In [3]:
import numpy as np
import pandas as pd
loan=pd.read_csv('loan_result.csv')
loan_group=loan.dropna().groupby('application_id').sum()
loan_group['num']=1
loan_group.groupby('is_applied').sum()[['num']]

,num
is_applied,
0.0,351356
1.0,267622
2.0,71144
3.0,23310
4.0,8721
5.0,3593
6.0,1590
7.0,717
8.0,387


In [73]:
import pandas as pd
data = pd.read_csv("finaljoined.csv")

In [74]:
del data["Unnamed: 0"]

In [75]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
x = data.birth_year.astype(str)
x = x.str[:4]
data.birth_year = pd.to_datetime(x, format = "%Y")
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [22]:
import numpy as np
from catboost import CatBoost, Pool

In [76]:
income_dummy=pd.get_dummies(data['income_type'])

In [77]:
data=pd.concat([data,income_dummy],axis=1)

In [78]:
del data['income_type']

In [79]:
employment_dummy=pd.get_dummies(data['employment_type'])

data=pd.concat([data,employment_dummy],axis=1)

del data['employment_type']

In [80]:
house_dummy=pd.get_dummies(data['houseown_type'])

data=pd.concat([data,house_dummy],axis=1)

del data['houseown_type']

In [81]:
purpose_dummy=pd.get_dummies(data['purpose'])
purpose_dummy.rename(columns={"기타":"기타목적"},inplace=True)

data=pd.concat([data,purpose_dummy],axis=1)

del data['purpose']

In [82]:
train_data=data[data['insert_time']<'2022-05-16']

In [83]:
train_data2=train_data.copy()

In [84]:
del train_data['is_applied']

In [85]:
test_data=data[data['insert_time']>='2022-05-16']
test_data=test_data[test_data['insert_time']<'2022-06-01']

In [86]:
test_data2=test_data.copy()

In [87]:
del test_data['is_applied']

In [88]:
train_label=train_data2['is_applied']

In [89]:
test_label=test_data2['is_applied']

In [90]:
from catboost import CatBoostClassifier

In [99]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [100]:
model.fit(train_data,train_label)

0:	learn: 0.1839555	total: 1.81s	remaining: 1.81s
1:	learn: 0.1779653	total: 3.07s	remaining: 0us


In [119]:
preds_proba=model.predict_proba(test_data)

In [116]:
preds_proba

array([[0.92800132, 0.07199868],
       [0.98662833, 0.01337167],
       [0.98662833, 0.01337167],
       ...,
       [0.95363376, 0.04636624],
       [0.95363376, 0.04636624],
       [0.95363376, 0.04636624]])

In [120]:
preds_proba=preds_proba[:,1]

In [121]:
preds_proba

array([0.07199868, 0.01337167, 0.01337167, ..., 0.04636624, 0.04636624,
       0.04636624])

In [104]:
from sklearn.metrics import f1_score

In [128]:
f1_score(test_label,preds_proba>0.054)

0.22314963621153483

In [123]:
preds_class=model.predict(test_data)

In [124]:
preds_class

array([0., 0., 0., ..., 0., 0., 0.])

In [126]:
f1_score(test_label,preds_class)

0.010421390691088137